# T1485 - Data Destruction
Adversaries may destroy data and files on specific systems or in large numbers on a network to interrupt availability to systems, services, and network resources. Data destruction is likely to render stored data irrecoverable by forensic techniques through overwriting files or data on local and remote drives.(Citation: Symantec Shamoon 2012)(Citation: FireEye Shamoon Nov 2016)(Citation: Palo Alto Shamoon Nov 2016)(Citation: Kaspersky StoneDrill 2017)(Citation: Unit 42 Shamoon3 2018)(Citation: Talos Olympic Destroyer 2018) Common operating system file deletion commands such as <code>del</code> and <code>rm</code> often only remove pointers to files without wiping the contents of the files themselves, making the files recoverable by proper forensic methodology. This behavior is distinct from [Disk Content Wipe](https://attack.mitre.org/techniques/T1561/001) and [Disk Structure Wipe](https://attack.mitre.org/techniques/T1561/002) because individual files are destroyed rather than sections of a storage disk or the disk's logical structure.

Adversaries may attempt to overwrite files and directories with randomly generated data to make it irrecoverable.(Citation: Kaspersky StoneDrill 2017)(Citation: Unit 42 Shamoon3 2018) In some cases politically oriented image files have been used to overwrite data.(Citation: FireEye Shamoon Nov 2016)(Citation: Palo Alto Shamoon Nov 2016)(Citation: Kaspersky StoneDrill 2017)

To maximize impact on the target organization in operations where network-wide availability interruption is the goal, malware designed for destroying data may have worm-like features to propagate across a network by leveraging additional techniques like [Valid Accounts](https://attack.mitre.org/techniques/T1078), [OS Credential Dumping](https://attack.mitre.org/techniques/T1003), and [SMB/Windows Admin Shares](https://attack.mitre.org/techniques/T1021/002).(Citation: Symantec Shamoon 2012)(Citation: FireEye Shamoon Nov 2016)(Citation: Palo Alto Shamoon Nov 2016)(Citation: Kaspersky StoneDrill 2017)(Citation: Talos Olympic Destroyer 2018)

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/haresudhan/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Windows - Overwrite file with Sysinternals SDelete
Overwrites and deletes a file using Sysinternals SDelete. Upon successful execution, "Files deleted: 1" will be displayed in
the powershell session along with other information about the file that was deleted.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Secure delete tool from Sysinternals must exist on disk at specified location (#{sdelete_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path $env:TEMP\Sdelete\sdelete.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://download.sysinternals.com/files/SDelete.zip" -OutFile "$env:TEMP\SDelete.zip"
Expand-Archive $env:TEMP\SDelete.zip $env:TEMP\Sdelete -Force
Remove-Item $env:TEMP\SDelete.zip -Force

```
##### Description: The file to delete must exist at #{file_to_delete}

##### Check Prereq Commands:
```powershell
if (Test-Path $env:TEMP\T1485.txt) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```powershell
New-Item $env:TEMP\T1485.txt -Force | Out-Null

```

In [ ]:
Invoke-AtomicTest T1485 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-Expression -Command "$env:TEMP\Sdelete\sdelete.exe -accepteula $env:TEMP\T1485.txt"
```

In [ ]:
Invoke-AtomicTest T1485 -TestNumbers 1

### Atomic Test #2 - macOS/Linux - Overwrite file with DD
Overwrites and deletes a file using DD.
To stop the test, break the command with CTRL/CMD+C.

**Supported Platforms:** linux, macos
#### Attack Commands: Run with `bash`
```bash
dd of=/var/log/syslog if=/dev/zero
```

In [ ]:
Invoke-AtomicTest T1485 -TestNumbers 2

## Detection
Use process monitoring to monitor the execution and command-line parameters of binaries that could be involved in data destruction activity, such as [SDelete](https://attack.mitre.org/software/S0195). Monitor for the creation of suspicious files as well as high unusual file modification activity. In particular, look for large quantities of file modifications in user directories and under <code>C:\Windows\System32\</code>.